# Introduction

This notebook presents a benchmark comparison between `power-grid-model`, 
[`pandapower`](http://www.pandapower.org/), and [`OpenDSS`](https://www.epri.com/pages/sa/opendss).
It runs several calculations, measures the calculation time, and compares the results.

## Test Network

The test network is fictionally generated using pre-defined random criteria. It is a radial network as follows:

```

source --- source_node ---| ---line--- node ---line--- node ...   (n_node_per_feeder)
                          |              |              |
                          |            load            load ...
                          |
                          | ---line--- ...
                          | .
                          | .
                          | .
                          | (n_feeder)

```

There is a node which is connected to a source (external network). From the source node there are `n_feeder` feeders. For each feeder there are `n_node_per_feeder` nodes, lines, and asymmetric loads. There are in total `n_feeder * n_node_per_feeder + 1` nodes in the network.


## Calculation

The notebook runs a power flow calculation with the same input data in `power-grid-model`, `pandapower`, and `OpenDSS`. It runs the following calculations:

* Single calculation with solver initialization.
* Single calculation without solver initialization (using pre-cached internal matrices).
* Time-series calculation
* N-1 calculation (not for `OpenDSS`)

The calculation is run symmetrically (not for `OpenDSS`) and asymmetrically. Both `power-grid-model` and `pandapower` supports asymmetric loads in symmetric calculations: the three-phase load is aggregated into one symmetric load. In `OpenDSS` we only run asymmetric calculations 

We use the Newton-Raphson method for `power-grid-model` and `pandapower`. In addition, we also use the iterative current and linear method in `power-grid-model` to see how much performance you can gain in exchange for accuracy. For `OpenDSS` the default fix point method is used.

## Results Comparison

The following results are compared between `power-grid-model`, `pandapower`, and `OpenDSS`:

* Per unit voltage of nodes (buses). For asymmetric calculation, it compares the value per phase.
* Loading of the lines. (not for `OpenDSS`)

It only compares the results of the Newton-Raphson in `pandapower` and `power-grid-model`, as well as the fix point method in `OpenDSS`, since the linear method of `power-grid-model` will produce a less accurate result.

## Performance Benchmark

The CPU time is measured for the **calculation part** of the program. The data preparation and model initialization is not measured. Furthermore, the single calculation is benchmarked with and without solver initialization. The former needs to execute connectivity check and initialize internal matrices (for example node admittance matrix). The latter uses the pre-cached connectivity and internal matrices.

# Preparation

## Import Libraries

We import neede libraries here. The fictional network generation and time-series profile generation is in a different Python file
[generate_fictional_dataset.py](./generate_fictional_dataset.py).

In [1]:
import enum
import timeit
from pathlib import Path
from copy import deepcopy
from dataclasses import dataclass
from typing import Any
import numba

import numpy as np
import pandapower as pp
import pandas as pd
import power_grid_model as pgm
from power_grid_model.data_types import SingleDataset
import lightsim2grid as l2g

from generate_fictional_dataset import generate_fictional_grid, LightSim2GridNetInput

from dss import DSS as dss_engine
import grid2op as g2o
from grid2op.Backend import PandaPowerBackend
from grid2op.Chronics import ChangeNothing, FromHandlers, GridStateFromFile
from grid2op.Chronics.handlers import CSVHandler, DoNothingHandler
from lightsim2grid.lightSimBackend import LightSimBackend
from lightsim2grid.timeSerie import TimeSerie

import warnings

warnings.filterwarnings("ignore")

c:\Users\MartijnGovers\Documents\tmp\power-grid-model-benchmark\venv\Lib\site-packages\grid2op\MakeEnv\Make.py:13: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


## Prepare Tables

The performance comparison table and result deviation table is initialized below.

In [2]:
class Method(enum.StrEnum):
    PGM_LINEAR_IMPEDANCE = "PGM Linear Impedance"
    PGM_LINEAR_CURRENT = "PGM Linear Current"
    PGM_ITERATIVE_CURRENT = "PGM Iterative Current"
    PGM_NEWTON_RAPHSON = "PGM Newton-Raphson"
    PANDAPOWER = "PandaPower Newton-Raphson"
    OPENDSS = "OpenDSS Fix Point"
    LIGHTSIM2GRID = "LightSim2Grid direct (NR)"
    GRID2OP_PANDAPOWER = "Grid2Op w/ PandaPower backend (NR)"
    GRID2OP_LIGHTSIM2GRID = "Grid2Op w/ LightSim2Grid backend (NR)"


class Calculation(enum.StrEnum):
    SYMMETRIC = "Symmetric calculation"
    ASYMMETRIC = "Asymmetric calculation"
    SYMMETRIC_WITH_SOLVER_INIT = "Symmetric calculation with solver initialization"
    SYMMETRIC_WITHOUT_SOLVER_INIT = (
        "Symmetric calculation without solver initialization"
    )
    ASYMMETRIC_WITH_SOLVER_INIT = "Asymmetric calculation with solver initialization"
    ASYMMETRIC_WITHOUT_SOLVER_INIT = (
        "Asymmetric calculation without solver initialization"
    )
    TIME_SERIES_SYMMETRIC = "Time series symmetric calculation"
    TIME_SERIES_ASYMMETRIC = "Time series asymmetric calculation"
    N_MINUS_1_SYMMETRIC = "N-1 symmetric calculation"
    N_MINUS_1_ASYMMETRIC = "N-1 asymmetric calculation"


# summary
summary_df = pd.DataFrame(
    np.full(shape=(8, 9), dtype=np.float64, fill_value=np.inf),
    columns=list(method.value for method in Method),
    index=[
        Calculation.SYMMETRIC_WITH_SOLVER_INIT.value,
        Calculation.SYMMETRIC_WITHOUT_SOLVER_INIT.value,
        Calculation.ASYMMETRIC_WITH_SOLVER_INIT.value,
        Calculation.ASYMMETRIC_WITHOUT_SOLVER_INIT.value,
        Calculation.TIME_SERIES_SYMMETRIC.value,
        Calculation.TIME_SERIES_ASYMMETRIC.value,
        Calculation.N_MINUS_1_SYMMETRIC.value,
        Calculation.N_MINUS_1_ASYMMETRIC.value,
    ],
)


class GridObservable(enum.StrEnum):
    VOLTAGE = "Voltage (p.u.)"
    LOADING = "Loading (p.u.)"


comparison_dict = {
    Method.PANDAPOWER: {
        GridObservable.VOLTAGE: "Deviation Voltage (p.u.) PandaPower",
        GridObservable.LOADING: "Deviation Loading (p.u.) PandaPower",
    },
    Method.OPENDSS: {
        GridObservable.VOLTAGE: "Deviation Voltage (p.u.) OpenDSS",
    },
    Method.GRID2OP_PANDAPOWER: {
        GridObservable.VOLTAGE: "Deviation Voltage (p.u.) Grid2Op w/ PP backend",
        GridObservable.LOADING: "Deviation Loading (p.u.) Grid2Op w/ PP backend",
    },
    Method.GRID2OP_LIGHTSIM2GRID: {
        GridObservable.VOLTAGE: "Deviation Voltage (p.u.) Grid2Op w/ LightSim2Grid backend",
        GridObservable.LOADING: "Deviation Loading (p.u.) Grid2Op w/ LightSim2Grid backend",
    },
}


comparison_df = pd.DataFrame(
    np.full(shape=(6, 7), dtype=np.float64, fill_value=np.nan),
    columns=list(
        comparison_type
        for method in Method
        for comparison_type in comparison_dict.get(method, {}).values()
    ),
    index=[
        Calculation.SYMMETRIC.value,
        Calculation.ASYMMETRIC.value,
        Calculation.TIME_SERIES_SYMMETRIC.value,
        Calculation.TIME_SERIES_ASYMMETRIC.value,
        Calculation.N_MINUS_1_SYMMETRIC.value,
        Calculation.N_MINUS_1_ASYMMETRIC.value,
    ],
)

def add_to_summary(calculation: Calculation, method: Method, execution_time: float):
    summary_df.loc[calculation.value, method.value] = execution_time

def add_to_comparison(calculation: Calculation, method: Method, observable: GridObservable, deviation: float):
    comparison_df.loc[calculation.value, comparison_dict[method][observable]] = deviation


## Simulation Parameters

The simulation parameters, for example, the total number of feeders `n_feeder`, are defined below.

In [3]:
# fictional grid parameters

n_node_per_feeder = 10
n_feeder = 100

cable_length_km_min = 0.8
cable_length_km_max = 1.2
load_p_w_max = 0.4e6 * 0.8
load_p_w_min = 0.4e6 * 1.2
pf = 0.95

load_scaling_min = 0.5
load_scaling_max = 1.5
n_step = 1000

# benchmark parameters
use_lightsim2grid = True
# when running a single scenario, repeat this many times to get a good estimate of the average time. NOTE: not needed for update data, because it already contains many scenarios.
n_single_scenario_repeats = 5

In [4]:
# override small network

# n_node_per_feeder = 3
# n_feeder = 2
# n_step = 10

In [5]:
# derived values

n_node = n_node_per_feeder * n_feeder + 1
n_line = n_node_per_feeder * n_feeder
n_load = n_node_per_feeder * n_feeder

## Pre-cache Library

To make a fair comparison, we run one small network so that `pandapower` can cache their dependent libraries into the memory.

* For `pandapower` the `numba` functions are JIT compiled and cached in the memory.

In [6]:
fictional_dataset = generate_fictional_grid(
    n_node_per_feeder=3,
    n_feeder=2,
    cable_length_km_min=cable_length_km_min,
    cable_length_km_max=cable_length_km_max,
    load_p_w_max=load_p_w_max,
    load_p_w_min=load_p_w_min,
    pf=pf,
    n_step=n_step,
    load_scaling_min=load_scaling_min,
    load_scaling_max=load_scaling_max,
)

pp.runpp(fictional_dataset['pp_net'], algorithm='nr', calculate_voltage_angles=True, distributed_slack=True, 
         lightsim2grid=use_lightsim2grid)
pgm_model = pgm.PowerGridModel(fictional_dataset['pgm_dataset'])
pgm_result = pgm_model.calculate_power_flow()

# Generate Dataset

First generate the fictional datasets.

In [7]:
fictional_dataset = generate_fictional_grid(
    n_node_per_feeder=n_node_per_feeder,
    n_feeder=n_feeder,
    cable_length_km_min=cable_length_km_min,
    cable_length_km_max=cable_length_km_max,
    load_p_w_max=load_p_w_max,
    load_p_w_min=load_p_w_min,
    pf=pf,
    n_step=n_step,
    load_scaling_min=load_scaling_min,
    load_scaling_max=load_scaling_max,
)

pp_net = deepcopy(fictional_dataset["pp_net"])
pgm_dataset = fictional_dataset["pgm_dataset"]
pgm_update_dataset = fictional_dataset["pgm_update_dataset"]
pp_time_series_dataset = fictional_dataset["pp_time_series_dataset"]
l2g_input = fictional_dataset["l2g_input"]

In [8]:
# initialize DSS

dss_engine.ClearAll()
dss_engine.Text.Command = f"compile {fictional_dataset['dss_file']}"

# Benchmark Function

In [9]:
@dataclass
class Execution:
    execution_time: float
    result: Any


def benchmark_return_time_and_result(
    func,
    *,
    is_batch: bool,
    setup="pass",
):
    result = None

    def _func():
        nonlocal result
        result = func()

    execution_time = timeit.repeat(
        _func,
        setup=setup,
        repeat=1 if is_batch else n_single_scenario_repeats,
        number=1,
    )
    return Execution(execution_time=np.mean(execution_time), result=result)


def benchmark_and_return_result(
    func,
    *,
    method: Method,
    calculation: Calculation,
    is_batch: bool,
    setup="pass",
):
    execution = benchmark_return_time_and_result(
        func=func, is_batch=is_batch, setup=setup
    )
    add_to_summary(calculation=calculation, method=method, execution_time=execution.execution_time)

    return execution.result


def benchmark_with_other_timer_and_return_result(
    func,
    *,
    timer,
    method: Method,
    calculation: Calculation,
    is_batch: bool,
    setup="pass",
):
    execution = benchmark_return_time_and_result(
        func=func, is_batch=is_batch, setup=setup
    )

    execution_time = timer()
    add_to_summary(calculation=calculation, method=method, execution_time=execution_time)

    return execution.result


def benchmark_pgm_power_flow(
    symmetric: bool,
    calculation: Calculation,
    update_data=None,
    with_initialization: bool = False,
):
    pgm_methods = {
        pgm.CalculationMethod.linear: Method.PGM_LINEAR_IMPEDANCE,
        pgm.CalculationMethod.linear_current: Method.PGM_LINEAR_CURRENT,
        pgm.CalculationMethod.iterative_current: Method.PGM_ITERATIVE_CURRENT,
        pgm.CalculationMethod.newton_raphson: Method.PGM_NEWTON_RAPHSON,
    }
    result = None
    for pgm_method, benchmark_method in pgm_methods.items():
        model_instance = None

        def _setup():
            # reset
            nonlocal model_instance
            model_instance = pgm.PowerGridModel(pgm_dataset)
            # cache internal state if we do not benchmark solver intialization
            if not with_initialization:
                model_instance.calculate_power_flow(
                    symmetric=symmetric, calculation_method=pgm_method
                )

        def _run():
            nonlocal model_instance
            return model_instance.calculate_power_flow(
                symmetric=symmetric,
                calculation_method=pgm_method,
                update_data=update_data,
                output_component_types={"node": ["u_pu"], "line": ["loading"]},
            )

        result = benchmark_and_return_result(
            _run,
            setup=_setup,
            method=benchmark_method,
            calculation=calculation,
            is_batch=update_data is not None,
        )
    return result


class TimedFunc:
    def __init__(self, func):
        self.func = func
        self.run_times = []

    def __call__(self, *args, **kwargs):
        """Decorator to time a function."""
        execution = benchmark_return_time_and_result(
            lambda: self.func(*args, **kwargs), is_batch=False
        )
        self.run_times.append(execution.execution_time)
        return execution.result

    def last_duration(self):
        """Return the last duration."""
        return self.run_times[-1] if self.run_times else None

    def total_duration(self):
        """Return the total duration."""
        return sum(self.run_times)

    def reset(self):
        self.run_times = []


class Runner:
    def __init__(self, grid, runner, resetter):
        self.orig_grid = grid
        self.runner = runner
        self.resetter = resetter
        self.reset()

    def reset(self):
        self.grid = self.resetter(self.orig_grid)

    def __call__(self):
        self.runner(self.grid)

# Single Calculation

We begin with a single power flow calculation.

## Symmetric

### power-grid-model

In [10]:
benchmark_pgm_power_flow(symmetric=True, calculation=Calculation.SYMMETRIC_WITH_SOLVER_INIT, with_initialization=True)
pgm_result = benchmark_pgm_power_flow(symmetric=True, calculation=Calculation.SYMMETRIC_WITHOUT_SOLVER_INIT, with_initialization=False)

### Newton-Raphson Method of pandapower

In [11]:
def run_pp(net):
    pp.runpp(
        net,
        algorithm="nr",
        calculate_voltage_angles=True,
        distributed_slack=True,
        lightsim2grid=use_lightsim2grid,
    )


pp_runner = Runner(
    pp_net,
    lambda x: run_pp(x),
    deepcopy,
)

benchmark_and_return_result(
    func=pp_runner,
    setup=lambda: pp_runner.reset(),
    method=Method.PANDAPOWER,
    calculation=Calculation.SYMMETRIC_WITH_SOLVER_INIT,
    is_batch=False,
)

# second calculation with existing
pp_runner()
benchmark_and_return_result(
    pp_runner,
    method=Method.PANDAPOWER,
    calculation=Calculation.SYMMETRIC_WITHOUT_SOLVER_INIT,
    is_batch=False,
)
run_pp(pp_net) # actually produce the output without resetting

### lightsim2grid

Similar to `pp.runpp(..., lightsim2grid=True)` but without the overhead of converting the pandapower grid to lightsim2grid input.

In [12]:
# first calculation with solver initialization
ls2g_runner = Runner(
    l2g_input,
    lambda l2g_input: l2g.newtonpf.newtonpf(
        **vars(l2g_input), options={"max_iteration": 20, "tolerance_mva": 1e-8}
    ),
    lambda x: LightSim2GridNetInput.copy_from(x),
)

benchmark_and_return_result(
    func=ls2g_runner,
    setup=lambda: ls2g_runner.reset(),
    method=Method.LIGHTSIM2GRID,
    calculation=Calculation.SYMMETRIC_WITH_SOLVER_INIT,
    is_batch=False,
)

# second calculation with existing
ls2g_runner()
benchmark_and_return_result(
    ls2g_runner,
    method=Method.LIGHTSIM2GRID,
    calculation=Calculation.SYMMETRIC_WITHOUT_SOLVER_INIT,
    is_batch=False,
)

### Grid2Op with PandaPower backend

In [13]:
pp_backend = None
g2o_pp_grid = None


def setup_g2o_pp():
    global pp_backend
    pp_backend = PandaPowerBackend(lightsim2grid=use_lightsim2grid, dist_slack=True)
    pp_backend.apply_action = TimedFunc(pp_backend.apply_action)
    pp_backend.runpf = TimedFunc(pp_backend.runpf)


def run_g2o_pp():
    global g2o_pp_grid
    g2o_pp_grid = g2o.make(
        Path("g2o_grid_sym").absolute(),
        backend=pp_backend,
        data_feeding_kwargs={
            "gridvalueClass": ChangeNothing,
        },
    )


def time_g2o_pp():
    return pp_backend.apply_action.total_duration() + pp_backend.runpf.total_duration()


def reset_g2o_pp():
    pp_backend.apply_action.reset()
    pp_backend.runpf.reset()


benchmark_with_other_timer_and_return_result(
    run_g2o_pp,
    timer=time_g2o_pp,
    setup=setup_g2o_pp,
    calculation=Calculation.SYMMETRIC_WITH_SOLVER_INIT,
    method=Method.GRID2OP_PANDAPOWER,
    is_batch=False,
)

_ = benchmark_with_other_timer_and_return_result(
    lambda: pp_backend.runpf(),
    timer=time_g2o_pp,
    setup=reset_g2o_pp,
    calculation=Calculation.SYMMETRIC_WITH_SOLVER_INIT,
    method=Method.GRID2OP_PANDAPOWER,
    is_batch=False,
)

### Grid2Op with LightSim2Grid backend

In [14]:
ls2g_backend = None
g2o_ls2g_grid = None


def setup_g2o_ls2g():
    global ls2g_backend
    ls2g_backend = LightSimBackend()
    ls2g_backend.apply_action = TimedFunc(
        ls2g_backend.apply_action
    )  # updates, recalculates topo, etc.
    ls2g_backend.runpf = TimedFunc(ls2g_backend.runpf)


def run_g2o_ls2g():
    global g2o_ls2g_grid
    g2o_ls2g_grid = g2o.make(
        Path("g2o_grid_sym").absolute(),
        backend=ls2g_backend,
        data_feeding_kwargs={
            "gridvalueClass": ChangeNothing,
        },
    )


def time_g2o_ls2g():
    return ls2g_backend.apply_action.total_duration() + ls2g_backend.runpf.total_duration()


def reset_g2o_pp():
    pp_backend.apply_action.reset()
    pp_backend.runpf.reset()


benchmark_with_other_timer_and_return_result(
    run_g2o_ls2g,
    timer=time_g2o_ls2g,
    setup=setup_g2o_ls2g,
    calculation=Calculation.SYMMETRIC_WITH_SOLVER_INIT,
    method=Method.GRID2OP_LIGHTSIM2GRID,
    is_batch=False,
)

_ = benchmark_with_other_timer_and_return_result(
    func=lambda: ls2g_backend.runpf(),
    timer=time_g2o_ls2g,
    setup=reset_g2o_pp,
    calculation=Calculation.SYMMETRIC_WITH_SOLVER_INIT,
    method=Method.GRID2OP_LIGHTSIM2GRID,
    is_batch=False,
)

### Calculate Deviation for Newton-Raphson Method

In [15]:
def get_g2o_u_pu_difference(
    g2o_obs: g2o.Observation.BaseObservation, pgm_scenario: SingleDataset
):
    # we can't use rho here because it is defined on the from-side but the to-side may also larger. PGM handles that, but grid2op does not.
    g2o_voltage_source = np.abs(g2o_obs.gen_v)
    g2o_voltage_loads = np.abs(g2o_obs.load_v)
    g2o_voltage = np.concatenate((g2o_voltage_source, g2o_voltage_loads), axis=0)
    return np.abs(
        (g2o_voltage * 1e3 / pgm_dataset["node"]["u_rated"])
        - pgm_scenario["node"]["u_pu"]
    ).max()


def get_g2o_loading_difference(
    g2o_obs: g2o.Observation.BaseObservation, pgm_scenario: SingleDataset
):
    # we can't use rho here because it is defined on the from-side but the to-side may also larger. PGM handles that, but grid2op does not.
    g2o_loading_from = np.abs(g2o_obs.a_or / g2o_obs.thermal_limit)
    g2o_loading_to = np.abs(g2o_obs.a_ex / g2o_obs.thermal_limit)
    g2o_loading = np.maximum(g2o_loading_from, g2o_loading_to)
    return np.abs(g2o_loading - pgm_scenario["line"]["loading"]).max()


add_to_comparison(
    calculation=Calculation.SYMMETRIC,
    method=Method.PANDAPOWER,
    observable=GridObservable.VOLTAGE,
    deviation=np.abs(pp_net.res_bus["vm_pu"] - pgm_result["node"]["u_pu"]).max(),
)
add_to_comparison(
    calculation=Calculation.SYMMETRIC,
    method=Method.PANDAPOWER,
    observable=GridObservable.LOADING,
    deviation=np.abs(
        pp_net.res_line["loading_percent"] * 1e-2 - pgm_result["line"]["loading"]
    ).max(),
)

# grid2op with pp backend
g2o_pp_obs = g2o_pp_grid.reset()
add_to_comparison(
    calculation=Calculation.SYMMETRIC,
    method=Method.GRID2OP_PANDAPOWER,
    observable=GridObservable.VOLTAGE,
    deviation=get_g2o_u_pu_difference(g2o_obs=g2o_pp_obs, pgm_scenario=pgm_result),
)
add_to_comparison(
    calculation=Calculation.SYMMETRIC,
    method=Method.GRID2OP_PANDAPOWER,
    observable=GridObservable.LOADING,
    deviation=get_g2o_loading_difference(g2o_obs=g2o_pp_obs, pgm_scenario=pgm_result),
)

# grid2op with lightsim2grid backend
g2o_ls2g_obs = g2o_ls2g_grid.reset()
add_to_comparison(
    calculation=Calculation.SYMMETRIC,
    method=Method.GRID2OP_LIGHTSIM2GRID,
    observable=GridObservable.VOLTAGE,
    deviation=get_g2o_u_pu_difference(g2o_obs=g2o_ls2g_obs, pgm_scenario=pgm_result),
)
add_to_comparison(
    calculation=Calculation.SYMMETRIC,
    method=Method.GRID2OP_LIGHTSIM2GRID,
    observable=GridObservable.LOADING,
    deviation=get_g2o_loading_difference(g2o_obs=g2o_ls2g_obs, pgm_scenario=pgm_result),
)

## Asymmetric

### power-grid-model

In [16]:
benchmark_pgm_power_flow(symmetric=False, calculation=Calculation.ASYMMETRIC_WITH_SOLVER_INIT, with_initialization=True)
pgm_result = benchmark_pgm_power_flow(symmetric=False, calculation=Calculation.ASYMMETRIC_WITHOUT_SOLVER_INIT, with_initialization=False)

### Newton-Raphson Method of pandapower

In [17]:
def run_pp_3ph(net):
    pp.runpp_3ph(
        net,
        algorithm="nr",
        calculate_voltage_angles=True,
        distributed_slack=True,
        lightsim2grid=use_lightsim2grid,
    )


pp_runner = Runner(
    pp_net,
    lambda x: run_pp_3ph(x),
    deepcopy,
)

benchmark_and_return_result(
    func=pp_runner,
    setup=lambda: pp_runner.reset(),
    method=Method.PANDAPOWER,
    calculation=Calculation.ASYMMETRIC_WITH_SOLVER_INIT,
    is_batch=False,
)

# second calculation with existing
pp_runner()
benchmark_and_return_result(
    pp_runner,
    method=Method.PANDAPOWER,
    calculation=Calculation.ASYMMETRIC_WITHOUT_SOLVER_INIT,
    is_batch=False,
)
run_pp_3ph(pp_net)  # actually produce the output without resetting

### Fix Point Method of OpenDSS

In [18]:
# first calculation with solver initialization
def setup_dss():
    dss_engine.ClearAll()
    dss_engine.Text.Command = f"compile {fictional_dataset['dss_file']}"


def run_dss_without_solver_init():
    dss_engine.ActiveCircuit.Solution.Solve()


def run_dss_with_solver_init():
    dss_engine.Text.Command = "set mode=snapshot"
    dss_engine.Text.Command = "set controlmode=static"
    run_dss_without_solver_init()


def setup_dss_with_init():
    setup_dss()
    run_dss_with_solver_init()

benchmark_and_return_result(
    run_dss_with_solver_init,
    setup=setup_dss,
    method=Method.OPENDSS,
    calculation=Calculation.ASYMMETRIC_WITH_SOLVER_INIT,
    is_batch=True,
)
benchmark_and_return_result(
    run_dss_without_solver_init,
    setup=setup_dss_with_init,
    method=Method.OPENDSS,
    calculation=Calculation.ASYMMETRIC_WITHOUT_SOLVER_INIT,
    is_batch=True,
)

### Calculate Deviation for Newton-Raphson Method

In [19]:
add_to_comparison(
    calculation=Calculation.ASYMMETRIC,
    method=Method.PANDAPOWER,
    observable=GridObservable.VOLTAGE,
    deviation=np.abs(
        pp_net.res_bus_3ph[["vm_a_pu", "vm_b_pu", "vm_c_pu"]].to_numpy()
        - pgm_result["node"]["u_pu"]
    ).max(),
)
add_to_comparison(
    calculation=Calculation.ASYMMETRIC,
    method=Method.PANDAPOWER,
    observable=GridObservable.LOADING,
    deviation=np.abs(
        pp_net.res_line_3ph["loading_percent"] * 1e-2 - pgm_result["line"]["loading"]
    ).max(),
)
add_to_comparison(
    calculation=Calculation.ASYMMETRIC,
    method=Method.OPENDSS,
    observable=GridObservable.VOLTAGE,
    deviation=np.abs(
        dss_engine.ActiveCircuit.AllBusVmagPu.reshape(-1, 3)
        - pgm_result["node"]["u_pu"]
    ).max(),
)

# Time Series Calculation

We execute a time-series power flow with `n_step` timestamps. 

## Preparation

The load profile is randomly generated by the function `generate_time_series`. It produces the relevant input format for both libraries.

In [20]:
time_steps = np.arange(n_step)

for x, y in zip(['p', 'q'], ['mw', 'mvar']):
    for p in ['a', 'b', 'c']:
        name = f'{x}_{p}_{y}'
        pp.control.ConstControl(
            pp_net,
            element='asymmetric_load',
            element_index=pp_net.asymmetric_load.index,
            variable=name,
            data_source=pp_time_series_dataset[name],
            profile_name=pp_net.asymmetric_load.index
        )

## Symmetric

### power-grid-model

In [21]:
pgm_result = benchmark_pgm_power_flow(
    symmetric=True,
    calculation=Calculation.TIME_SERIES_SYMMETRIC,
    update_data=pgm_update_dataset,
)

### Newton-Raphson Method of pandapower

In [22]:
def setup_pp_timeseries():
    if hasattr(pp_net, 'output_writer'):
        del pp_net.output_writer

    pp.timeseries.OutputWriter(
        pp_net,
        log_variables=[
            ('res_bus', 'vm_pu'),
            ('res_line', 'loading_percent'),
        ]
    )

def run_pp_timeseries():
    pp.timeseries.run_timeseries(
        pp_net, run=pp.runpp, time_steps=time_steps,
        calculate_voltage_angles=True, distributed_slack=True, lightsim2grid=use_lightsim2grid,
        verbose=False
    )

benchmark_and_return_result(
    func=run_pp_timeseries,
    setup=setup_pp_timeseries,
    method=Method.PANDAPOWER,
    calculation=Calculation.TIME_SERIES_SYMMETRIC,
    is_batch=True,
)

### Grid2op timeseries with PandaPower backend

Grid2op uses a Python outer loop by using `simulate()` to return a new observer at some timestep.

In [23]:
pp_backend = None
g2o_pp_sim_obs = None


def setup_g2o_pp_timeseries():
    global pp_backend
    global g2o_pp_grid

    pp_backend = PandaPowerBackend(lightsim2grid=use_lightsim2grid, dist_slack=True)
    pp_backend.apply_action = TimedFunc(
        pp_backend.apply_action
    )  # updates, recalculates topo, etc.
    pp_backend.runpf = TimedFunc(pp_backend.runpf)
    g2o_pp_grid = g2o.make(
        Path("g2o_grid_sym").absolute(),
        backend=pp_backend,
        data_feeding_kwargs={
            "gridvalueClass": FromHandlers,
            "load_p_handler": CSVHandler(array_name="load_p"),
            "load_q_handler": CSVHandler(array_name="load_q"),
            "gen_p_handler": DoNothingHandler(),
            "gen_v_handler": DoNothingHandler(),
        },
    )

    pp_backend.apply_action.reset()
    pp_backend.runpf.reset()


def run_g2o_pp_timeseries():
    global g2o_pp_sim_obs
    action = g2o_pp_grid.action_space()
    g2o_pp_sim_obs = [g2o_pp_grid.reset()] + [
        g2o_pp_grid.step(action)[0] for _ in range(n_step - 1)
    ]


def time_g2o_pp():
    return pp_backend.apply_action.total_duration() + pp_backend.runpf.total_duration()


_ = benchmark_with_other_timer_and_return_result(
    run_g2o_pp_timeseries,
    timer=time_g2o_pp,
    setup=setup_g2o_pp_timeseries,
    calculation=Calculation.TIME_SERIES_SYMMETRIC,
    method=Method.GRID2OP_PANDAPOWER,
    is_batch=True,
)

### Grid2Op timeseries with LightSim2Grid backend

In [24]:
ls2g_backend = None
g2o_ls2g_sim_obs = None


def setup_g2o_ls2g_timeseries():
    global ls2g_backend
    global g2o_ls2g_grid

    ls2g_backend = LightSimBackend()
    ls2g_backend.apply_action = TimedFunc(
        ls2g_backend.apply_action
    )  # updates, recalculates topo, etc.
    ls2g_backend.runpf = TimedFunc(ls2g_backend.runpf)
    g2o_ls2g_grid = g2o.make(
        Path("g2o_grid_sym").absolute(),
        backend=ls2g_backend,
        data_feeding_kwargs={
            "gridvalueClass": FromHandlers,
            "load_p_handler": CSVHandler(array_name="load_p"),
            "load_q_handler": CSVHandler(array_name="load_q"),
            "gen_p_handler": DoNothingHandler(),
            "gen_v_handler": DoNothingHandler(),
        },
    )

    ls2g_backend.apply_action.reset()
    ls2g_backend.runpf.reset()


def run_g2o_ls2g_timeseries():
    global g2o_ls2g_sim_obs
    action = g2o_ls2g_grid.action_space()
    g2o_ls2g_sim_obs = [g2o_ls2g_grid.reset()] + [
        g2o_ls2g_grid.step(action)[0] for _ in range(n_step - 1)
    ]


def time_g2o_ls2g():
    return (
        ls2g_backend.apply_action.total_duration() + ls2g_backend.runpf.total_duration()
    )


_ = benchmark_with_other_timer_and_return_result(
    run_g2o_ls2g_timeseries,
    timer=time_g2o_ls2g,
    setup=setup_g2o_ls2g_timeseries,
    calculation=Calculation.TIME_SERIES_SYMMETRIC,
    method=Method.GRID2OP_LIGHTSIM2GRID,
    is_batch=True,
)

### LightSim2Grid TimeSerie

NOTE: this feature only supports PV nodes, and therefore the results will differ from the PGM result.

This benchmark is still included as it gives a good idea of what can be.

In [25]:
ls2g_timeseries_backend = LightSimBackend()
g2o_ls2g_grid = g2o.make(
    Path("g2o_grid_sym_pv").absolute(),
    backend=ls2g_timeseries_backend,
    data_feeding_kwargs={
        "gridvalueClass": GridStateFromFile,
    },
)
time_series = TimeSerie(g2o_ls2g_grid)
time_series.get_flows = TimedFunc(time_series.get_flows)
res_p, res_a, res_v = benchmark_and_return_result(
    time_series.get_flows,
    calculation=Calculation.TIME_SERIES_SYMMETRIC,
    method=Method.LIGHTSIM2GRID,
    is_batch=True,
)

### Calculate Deviation for Newton-Raphson Method

In [26]:
from power_grid_model.utils import get_dataset_scenario

# pandapower
pp_u_pu = pp_net.output_writer.iloc[0, 0].output["res_bus.vm_pu"].to_numpy()
add_to_comparison(
    calculation=Calculation.TIME_SERIES_SYMMETRIC,
    method=Method.PANDAPOWER,
    observable=GridObservable.VOLTAGE,
    deviation=np.abs(pp_u_pu - pgm_result["node"]["u_pu"]).max(),
)
pp_loading = (
    pp_net.output_writer.iloc[0, 0].output["res_line.loading_percent"].to_numpy() * 1e-2
)
add_to_comparison(
    calculation=Calculation.TIME_SERIES_SYMMETRIC,
    method=Method.PANDAPOWER,
    observable=GridObservable.LOADING,
    deviation=np.abs(pp_loading - pgm_result["line"]["loading"]).max(),
)

# g2o w/ pp backend
add_to_comparison(
    calculation=Calculation.TIME_SERIES_SYMMETRIC,
    method=Method.GRID2OP_PANDAPOWER,
    observable=GridObservable.VOLTAGE,
    deviation=max(
        get_g2o_u_pu_difference(
            g2o_obs=obs, pgm_scenario=get_dataset_scenario(pgm_result, i)
        )
        for i, obs in enumerate(g2o_pp_sim_obs)
    ),
)
add_to_comparison(
    calculation=Calculation.TIME_SERIES_SYMMETRIC,
    method=Method.GRID2OP_PANDAPOWER,
    observable=GridObservable.LOADING,
    deviation=max(
        get_g2o_loading_difference(
            g2o_obs=obs, pgm_scenario=get_dataset_scenario(pgm_result, i)
        )
        for i, obs in enumerate(g2o_pp_sim_obs)
    ),
)

# g2o w/ ls2g backend
add_to_comparison(
    calculation=Calculation.TIME_SERIES_SYMMETRIC,
    method=Method.GRID2OP_LIGHTSIM2GRID,
    observable=GridObservable.VOLTAGE,
    deviation=max(
        get_g2o_u_pu_difference(
            g2o_obs=obs, pgm_scenario=get_dataset_scenario(pgm_result, i)
        )
        for i, obs in enumerate(g2o_ls2g_sim_obs)
    ),
)
add_to_comparison(
    calculation=Calculation.TIME_SERIES_SYMMETRIC,
    method=Method.GRID2OP_LIGHTSIM2GRID,
    observable=GridObservable.LOADING,
    deviation=max(
        get_g2o_loading_difference(
            g2o_obs=obs, pgm_scenario=get_dataset_scenario(pgm_result, i)
        )
        for i, obs in enumerate(g2o_ls2g_sim_obs)
    ),
)

# ls2g timeseries can't really be compared because it only supports PV nodes

## Asymmetric

### power-grid-model

In [27]:
pgm_result = benchmark_pgm_power_flow(symmetric=False, calculation=Calculation.TIME_SERIES_ASYMMETRIC, update_data=pgm_update_dataset)


### Newton-Raphson Method of pandapower

In [28]:
def setup_pp_timeseries_3ph():
    if hasattr(pp_net, "output_writer"):
        del pp_net.output_writer
    ow = pp.timeseries.OutputWriter(pp_net)
    ow.log_variable('res_bus_3ph', 'vm_a_pu', index=pp_net.bus.index)
    ow.log_variable('res_bus_3ph', 'vm_b_pu', index=pp_net.bus.index)
    ow.log_variable('res_bus_3ph', 'vm_c_pu', index=pp_net.bus.index)
    ow.log_variable('res_line_3ph', 'loading_percent', index=pp_net.line.index)

def run_pp_timeseries_3ph():
    pp.timeseries.run_timeseries(
        pp_net,
        run=pp.runpp_3ph,
        time_steps=time_steps,
        calculate_voltage_angles=True,
        distributed_slack=True,
        lightsim2grid=use_lightsim2grid,
        verbose=False
    )

benchmark_and_return_result(run_pp_timeseries_3ph, 
                            setup=setup_pp_timeseries_3ph,
                            method=Method.PANDAPOWER,
                            calculation=Calculation.TIME_SERIES_ASYMMETRIC,
                            is_batch=True)

### Fix Point Method of OpenDSS

In [29]:
# first calculation with solver initialization
def setup_dss_timeseries():
    dss_engine.ClearAll()
    dss_engine.Text.Command = f"compile {fictional_dataset['dss_file']}"

def run_dss_timeseries():
    dss_engine.Text.Command = "set mode=Daily"
    dss_engine.Text.Command = "set Stepsize=3600s"
    dss_engine.Text.Command = f"set Number={n_step}"
    dss_engine.Text.Command = "set controlmode=static"
    dss_engine.ActiveCircuit.Solution.Solve()

benchmark_and_return_result(
    run_dss_timeseries,
    setup=setup_dss_timeseries,
    method=Method.OPENDSS,
    calculation=Calculation.TIME_SERIES_ASYMMETRIC,
    is_batch=False,
)

In [30]:
# get results
all_results = []

# source
flag = dss_engine.ActiveCircuit.Monitors.First
assert flag != 0
all_results += [dss_engine.ActiveCircuit.Monitors.Channel(1), dss_engine.ActiveCircuit.Monitors.Channel(2), dss_engine.ActiveCircuit.Monitors.Channel(3)]

# others
flag = dss_engine.ActiveCircuit.Monitors.Next
while flag != 0:
    all_results.append(dss_engine.ActiveCircuit.Monitors.Channel(1))
    flag = dss_engine.ActiveCircuit.Monitors.Next

dss_voltage = np.stack(all_results, axis=1)
dss_voltage = dss_voltage.reshape(n_step, -1, 3)
dss_voltage /= 10e3 / np.sqrt(3)

### Calculate Deviation for Newton-Raphson Method

In [31]:
pp_u_pu = []
for p in ['a', 'b', 'c']:
    pp_u_pu.append(pp_net.output_writer.iloc[0, 0].output[f'res_bus_3ph.vm_{p}_pu'].to_numpy())
pp_u_pu = np.stack(pp_u_pu, axis=-1)
add_to_comparison(
    calculation=Calculation.TIME_SERIES_ASYMMETRIC,
    method=Method.PANDAPOWER,
    observable=GridObservable.VOLTAGE,
    deviation=np.abs(pp_u_pu - pgm_result['node']['u_pu']).max(),
)

pp_loading = pp_net.output_writer.iloc[0, 0].output[r'res_line_3ph.loading_percent'].to_numpy() * 1e-2
add_to_comparison(
    calculation=Calculation.TIME_SERIES_ASYMMETRIC,
    method=Method.PANDAPOWER,
    observable=GridObservable.LOADING,
    deviation=np.abs(pp_loading - pgm_result['line']['loading']).max(),
)

add_to_comparison(
    calculation=Calculation.TIME_SERIES_ASYMMETRIC,
    method=Method.OPENDSS,
    observable=GridObservable.VOLTAGE,
    deviation=np.abs(dss_voltage - pgm_result['node']['u_pu']).max(),
)

# N-1 Scenario Calculation

We execute a N-1 scenario calculation. There are `n_line` scenarios. In each scenario one line will be disabled. Since the original network is radial, part of the network will be unenergized due to the switch-off of a line.

## Preparation

The N-1 scenario input is generated for `power-grid-model`. Since `pandapower` does not have a built-in N-1 scenario calculation, we execute the power flow per scenario in a loop.

In [32]:
# re-generate dataset

pp_net = deepcopy(fictional_dataset["pp_net"])
pgm_dataset = fictional_dataset["pgm_dataset"]

# update dataset for power grid model
# disable one line per batch
pgm_line_profile = {
    "id": pgm_dataset["line"]["id"].reshape(-1, 1).copy(),
    "from_status": np.zeros((n_line, 1), dtype=np.int8),
    "to_status": np.zeros((n_line, 1), dtype=np.int8),
}
pgm_n_minus_1_update_dataset = {"line": pgm_line_profile}

## Symmetric

### power-grid-model

In [33]:
pgm_result = benchmark_pgm_power_flow(symmetric=True, calculation=Calculation.N_MINUS_1_SYMMETRIC, update_data=pgm_n_minus_1_update_dataset)

### Newton-Raphson Method of pandapower

In [34]:
# prepare pandapower result dataset
pp_u_pu = np.empty(shape=(n_line, n_node), dtype=np.float64)
pp_loading = np.empty(shape=(n_line, n_line), dtype=np.float64)

def run_pp_n_minus_1():
    # loop to calcualte pandapower N-1
    for i in pp_net.line.index:
        # set one line out of service
        pp_net.line.loc[i, 'in_service'] = False
        pp.runpp(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True, lightsim2grid=use_lightsim2grid)
        # restore that line
        pp_net.line.loc[i, 'in_service'] = True
        # get result
        pp_u_pu[i, :] = pp_net.res_bus['vm_pu']
        pp_loading[i, :] = pp_net.res_line['loading_percent'] * 1e-2

benchmark_and_return_result(run_pp_n_minus_1, method=Method.PANDAPOWER, calculation=Calculation.N_MINUS_1_SYMMETRIC, is_batch=True)

# set nan to 0.0 to make a meaningful comparison
pp_u_pu[np.isnan(pp_u_pu)] = 0.0
pp_loading[np.isnan(pp_loading)] = 0.0

### LightSim2Grid

While implemented, the calculations fail to converge. Above all, it does so without reporting any error messages. As a result, we cannot trust the data.

In [35]:
# from lightsim2grid.contingencyAnalysis import ContingencyAnalysis

# ls2g_timeseries_backend = LightSimBackend()
# g2o_ls2g_grid = g2o.make(
#     Path("g2o_grid_sym").absolute(),
#     backend=ls2g_timeseries_backend,
#     data_feeding_kwargs={
#         "gridvalueClass": FromHandlers,
#         "load_p_handler": CSVHandler(array_name="load_p"),
#         "load_q_handler": CSVHandler(array_name="load_q"),
#         "gen_p_handler": DoNothingHandler(),
#         "gen_v_handler": DoNothingHandler(),
#     },
# )
# contingency_analysis = ContingencyAnalysis(g2o_ls2g_grid)
# contingency_analysis.add_all_n1_contingencies()
# contingency_analysis.get_flows = TimedFunc(contingency_analysis.get_flows)
# res_p, res_a, res_v = contingency_analysis.get_flows()
# add_to_summary(
#     calculation=Calculation.N_MINUS_1_SYMMETRIC,
#     method=Method.LIGHTSIM2GRID,
#     execution_time=contingency_analysis.get_flows.total_duration(),
# )

### Calculate Deviation for Newton-Raphson Method

In [36]:
add_to_comparison(
    calculation=Calculation.N_MINUS_1_SYMMETRIC,
    method=Method.PANDAPOWER,
    observable=GridObservable.VOLTAGE,
    deviation=np.abs(pp_u_pu - pgm_result['node']['u_pu']).max(),
)

add_to_comparison(
    calculation=Calculation.N_MINUS_1_SYMMETRIC,
    method=Method.PANDAPOWER,
    observable=GridObservable.LOADING,
    deviation=np.abs(pp_loading - pgm_result['line']['loading']).max(),
)

## Asymmetric

### power-grid-model

In [37]:
pgm_result = benchmark_pgm_power_flow(symmetric=False, calculation=Calculation.N_MINUS_1_ASYMMETRIC, update_data=pgm_n_minus_1_update_dataset)


### Newton-Raphson Method of pandapower

In [38]:
%%capture

# prepare pandapower result dataset
pp_u_pu = np.empty(shape=(n_line, n_node, 3), dtype=np.float64)
pp_loading = np.empty(shape=(n_line, n_line), dtype=np.float64)

def run_pp_n_minus_1_3ph():
    # loop to calcualte pandapower N-1
    for i in pp_net.line.index:
        # set one line out of service
        pp_net.line.loc[i, 'in_service'] = False
        pp.runpp_3ph(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True, lightsim2grid=use_lightsim2grid)
        # restore that line
        pp_net.line.loc[i, 'in_service'] = True
        # get result
        pp_u_pu[i, ...] = pp_net.res_bus_3ph[['vm_a_pu', 'vm_b_pu', 'vm_c_pu']]
        pp_loading[i, :] = pp_net.res_line_3ph['loading_percent'] * 1e-2

benchmark_and_return_result(run_pp_n_minus_1_3ph, method=Method.PANDAPOWER, calculation=Calculation.N_MINUS_1_ASYMMETRIC, is_batch=True)

# set nan to 0.0 to make a meaningful comparison
pp_u_pu[np.isnan(pp_u_pu)] = 0.0
pp_loading[np.isnan(pp_loading)] = 0.0

### Calculate Deviation for Newton-Raphson Method

In [39]:
add_to_comparison(
    calculation=Calculation.N_MINUS_1_ASYMMETRIC,
    method=Method.PANDAPOWER,
    observable=GridObservable.VOLTAGE,
    deviation=np.abs(pp_u_pu - pgm_result['node']['u_pu']).max(),
)

add_to_comparison(
    calculation=Calculation.N_MINUS_1_ASYMMETRIC,
    method=Method.PANDAPOWER,
    observable=GridObservable.LOADING,
    deviation=np.abs(pp_loading - pgm_result['line']['loading']).max(),
)

# Summary

## Deviation of Results

Below is the table of deviation between the results from `power-grid-model`, `pandapower`, and `OpenDSS`. It matches to the order of `1e-6`. Note there are no comparisons for `OpenDSS` for symmetric calculations.

In [40]:
display(comparison_df)

,Deviation Voltage (p.u.) PandaPower,Deviation Loading (p.u.) PandaPower,Deviation Voltage (p.u.) OpenDSS,Deviation Voltage (p.u.) Grid2Op w/ PP backend,Deviation Loading (p.u.) Grid2Op w/ PP backend,Deviation Voltage (p.u.) Grid2Op w/ LightSim2Grid backend,Deviation Loading (p.u.) Grid2Op w/ LightSim2Grid backend
Symmetric calculation,1.626699e-12,5.113132e-13,NaN,1.523866e-07,1.469299e-08,9.306312e-08,2.176525e-08
Asymmetric calculation,5.797001e-10,8.192117e-09,5.401326e-09,NaN,NaN,NaN,NaN
Time series symmetric calculation,1.786793e-12,1.294465e-12,NaN,1.561140e-07,3.274137e-08,9.662283e-08,4.643363e-08
Time series asymmetric calculation,5.894705e-10,3.375893e-09,4.417026e-06,NaN,NaN,NaN,NaN
N-1 symmetric calculation,1.637801e-12,5.923873e-13,NaN,NaN,NaN,NaN,NaN
N-1 asymmetric calculation,5.797118e-10,8.192222e-09,NaN,NaN,NaN,NaN,NaN


## Performance Comparison

Below is the table of the measured time of all calculations. Note there are no comparisons for `OpenDSS` for symmetric calculations.

In [41]:
display(summary_df)

,PGM Linear Impedance,PGM Linear Current,PGM Iterative Current,PGM Newton-Raphson,PandaPower Newton-Raphson,OpenDSS Fix Point,LightSim2Grid direct (NR),Grid2Op w/ PandaPower backend (NR),Grid2Op w/ LightSim2Grid backend (NR)
Symmetric calculation with solver initialization,0.002226,0.002525,0.002529,0.003678,0.033096,inf,0.009091,0.040634,0.005568
Symmetric calculation without solver initialization,0.001057,0.000989,0.001379,0.003121,0.020869,inf,0.005885,inf,inf
Asymmetric calculation with solver initialization,0.003725,0.003498,0.005431,0.012945,0.578501,0.018977,inf,inf,inf
Asymmetric calculation without solver initialization,0.001842,0.001612,0.002822,0.009487,0.382334,0.001119,inf,inf,inf
Time series symmetric calculation,0.453385,0.472444,0.833141,2.241664,18.672568,inf,7.483918,44.272685,1.315406
Time series asymmetric calculation,2.033993,2.245182,3.696315,9.327486,353.371271,2.148159,inf,inf,inf
N-1 symmetric calculation,1.218864,1.450322,1.547798,2.737500,17.939382,inf,inf,inf,inf
N-1 asymmetric calculation,3.309507,3.629910,3.667657,8.756861,246.475811,inf,inf,inf,inf


In [42]:
relative_df = summary_df.div(summary_df[Method.PANDAPOWER.value], axis=0)
speedup_df = 1 / relative_df
display(speedup_df.style.format("{:0.2f}x"))

,PGM Linear Impedance,PGM Linear Current,PGM Iterative Current,PGM Newton-Raphson,PandaPower Newton-Raphson,OpenDSS Fix Point,LightSim2Grid direct (NR),Grid2Op w/ PandaPower backend (NR),Grid2Op w/ LightSim2Grid backend (NR)
Symmetric calculation with solver initialization,14.87x,13.11x,13.09x,9.00x,1.00x,0.00x,3.64x,0.81x,5.94x
Symmetric calculation without solver initialization,19.74x,21.11x,15.14x,6.69x,1.00x,0.00x,3.55x,0.00x,0.00x
Asymmetric calculation with solver initialization,155.32x,165.37x,106.52x,44.69x,1.00x,30.48x,0.00x,0.00x,0.00x
Asymmetric calculation without solver initialization,207.61x,237.23x,135.50x,40.30x,1.00x,341.61x,0.00x,0.00x,0.00x
Time series symmetric calculation,41.18x,39.52x,22.41x,8.33x,1.00x,0.00x,2.50x,0.42x,14.20x
Time series asymmetric calculation,173.73x,157.39x,95.60x,37.88x,1.00x,164.50x,0.00x,0.00x,0.00x
N-1 symmetric calculation,14.72x,12.37x,11.59x,6.55x,1.00x,0.00x,0.00x,0.00x,0.00x
N-1 asymmetric calculation,74.48x,67.90x,67.20x,28.15x,1.00x,0.00x,0.00x,0.00x,0.00x
